In [43]:
import numpy as np
import math
import copy
import datetime

### Description of is_row_present:
This function tests whether input_row (1D np array) is present as a row in input_2D_array (2D np array).
1. If yes, the index of input_row in input_2D_array is returned.
2. If not, -1 is returned.

This function is used in the function merge_two_decompositions.

In [3]:
def is_row_present(input_row, input_2D_array):
    row_num, col_num = input_2D_array.shape
    
    for i in range(row_num):
        if np.array_equal(input_row, input_2D_array[i, :]):
            return i
    
    return -1

### Description of merge_two_decompositions:
This function takes as inputs:
1. A decomposition of some $p_{0}$-TPM $P$: x_list (a list of positive real numbers) and A_array (np array of distinct BN matrices)
2. A decomposition of some $q_{0}$-TPM $Q$: y_list (a list of positive real numbers) and B_array (np array of distinct BN matrices)

The function outputs a decomposition of the $(p_{0} + q_{0})$-TPM $P + Q$:
1. M (the length of the merged decompositions)
2. w_list (list of the positive weights of the merged decompositions)
3. C_array (np array of distinct BN matrices of the merged decompositions)

In [4]:
def merge_two_decompositions(x_list, A_array, y_list, B_array):
    C_array = copy.deepcopy(A_array)
    w_list = copy.deepcopy(x_list)
    M = len(w_list)
    y_length = len(y_list)
    
    for i in range(y_length):
        position = is_row_present(B_array[i, :], C_array)
        
        if (position == -1):
            M += 1
            w_list.append(y_list[i])
            C_array = np.vstack((C_array, B_array[i, :]))
        else:
            w_list[position] += y_list[i]
    
    return M, w_list, C_array

### Description of add_up_a_decomposition
The function takes as inputs:
1. a decomposition (x_list, BN_matrices_array) output by SER 1, SER 2, GER (x_list is a list, whereas BN_matrices_array is a 2D np array).
2. The dimension of the output TPM row_dim, col_dim

The function then sums up (x_list, BN_matrices_array) and outputs the resulting TPM (row_dim-by-col_dim).

In [7]:
def add_up_a_decomposition(x_list, BN_matrices_array, row_dim, col_dim):
    output_TPM = np.zeros((row_dim, col_dim), dtype=int)
    num_BN_matrices = len(x_list)
    
    for k in range(num_BN_matrices):
        for col in range(col_dim):
            output_TPM[BN_matrices_array[k, col], col] += x_list[k]
    
    return output_TPM

## The Division Pre-processing Algorithm (D-Pre)

In [8]:
def division_preprocess(P, d):
    row_num, col_num = P.shape
    Q_tilde = np.zeros((row_num, col_num), dtype=int)
    R_tilde = np.zeros((row_num, col_num), dtype=int)
    
    for row in range(row_num):
        for col in range(col_num):
            Q_tilde[row, col] = P[row, col] // d
            R_tilde[row, col] = P[row, col] % d
    
    r_tilde = np.sum(R_tilde, axis=0)
    r_tilde_max = np.max(r_tilde)
    
    for j in range(col_num):
        i = 0
        while (r_tilde[j] < r_tilde_max):
            if (d * Q_tilde[i, j] < r_tilde_max - r_tilde[j]):
                R_tilde[i, j] += d * Q_tilde[i, j]
                r_tilde[j] += d * Q_tilde[i, j]
                Q_tilde[i, j] = 0
            else:
                R_tilde[i, j] += r_tilde_max - r_tilde[j]
                Q_tilde[i, j] -= (r_tilde_max - r_tilde[j]) // d
                r_tilde[j] = r_tilde_max
            i += 1
    
    return Q_tilde, R_tilde

## The Simple Entry Removal Algorithm 1 (SER 1)

In [33]:
# This function is used in the SER 1 function.
# residue_matrix_R is a np.ndarray.
# PBN_row_num is the number of rows present in residue_matrix_R.
# PBN_col_num is the number of columns present in residue_matrix_R.
def find_smallest_nonzero_entry(residue_matrix_R, PBN_row_num, PBN_col_num):
    row_counter = 0
    col_counter = 0
    while residue_matrix_R[row_counter, col_counter] == 0:
        row_counter += 1
    smallest_entry = residue_matrix_R[row_counter, col_counter]
    smallest_entry_row_pos = row_counter
    smallest_entry_col_pos = col_counter

    del row_counter
    del col_counter
    for col_counter in range(PBN_col_num):
        for row_counter in range(PBN_row_num):
            current_entry = residue_matrix_R[row_counter, col_counter]
            if (current_entry > 0) and (current_entry < smallest_entry):
                smallest_entry = current_entry
                smallest_entry_row_pos = row_counter
                smallest_entry_col_pos = col_counter

    return smallest_entry, smallest_entry_row_pos, smallest_entry_col_pos

In [34]:
def SER_1(PBN_matrix_P):  # input: 2^n-by-2^n ndarray consisting of integers.
    PBN_row_num = PBN_matrix_P.shape[0]
    PBN_col_num = PBN_matrix_P.shape[1]
    coefficient_list_xi = []  # the coefficients of BN matrices to be output
    # the BN matrices to be output
    BN_matrices_list_Ai = np.array([[-1] * PBN_col_num])  # the first row of BN_matrices_list_Ai will be
                                                          # replaced in the first iteration of pseudocode
                                                          # steps 1 to 4.
                                                          
    residue_matrix_R = PBN_matrix_P.copy()
    k = 1  # counts the number of iterations of pseudocode Steps 1 to 4
    chosen_entry_xi, xi_row_pos, xi_col_pos = find_smallest_nonzero_entry(residue_matrix_R, 
                                                                          PBN_row_num, PBN_col_num)
    chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
    chosen_entry_pos_each_col[0, xi_col_pos] = xi_row_pos
    
    coefficient_list_xi.append(chosen_entry_xi)
    BN_matrices_list_Ai = chosen_entry_pos_each_col
    
    for col_counter in range(PBN_col_num):
        row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
        residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    zero_PBN_matrix = np.zeros((PBN_row_num, PBN_col_num), dtype=int)
    
    while (not np.array_equal(residue_matrix_R, zero_PBN_matrix)):
        k += 1
        chosen_entry_xi, xi_row_pos, xi_col_pos = find_smallest_nonzero_entry(residue_matrix_R, 
                                                                              PBN_row_num, PBN_col_num)
        chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
        chosen_entry_pos_each_col[0, xi_col_pos] = xi_row_pos
    
        coefficient_list_xi.append(chosen_entry_xi)
        BN_matrices_list_Ai = np.append(BN_matrices_list_Ai, chosen_entry_pos_each_col, axis=0)
        
        for col_counter in range(PBN_col_num):
            row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
            residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    return k, coefficient_list_xi, BN_matrices_list_Ai

## The Simple Entry Removal Algorithm 2 (SER 2)

In [9]:
# This function is used in the SER 2 function.
# residue_matrix_R is a np.ndarray.
# chosen_entry_pos_each_col is a 1-by-PBN_col_sum np.ndarray.
# PBN_col_num is the number of columns present in residue_matrix_R

def choose_coefficient(residue_matrix_R, chosen_entry_pos_each_col, PBN_col_num):
    col_counter = 0
    chosen_entry_xi = residue_matrix_R[chosen_entry_pos_each_col[0, col_counter], col_counter]
    
    for col_counter in range(PBN_col_num):
        current_entry = residue_matrix_R[chosen_entry_pos_each_col[0, col_counter], col_counter]
        if current_entry < chosen_entry_xi:
            chosen_entry_xi = current_entry
    
    return chosen_entry_xi

In [10]:
def SER_2(PBN_matrix_P):  # input: 2^n-by-2^n ndarray consisting of integers.
    PBN_row_num = PBN_matrix_P.shape[0]
    PBN_col_num = PBN_matrix_P.shape[1]
    coefficient_list_xi = []  # the coefficients of BN matrices to be output
    # the BN matrices to be output
    BN_matrices_list_Ai = np.array([[-1] * PBN_col_num])  # the first row of BN_matrices_list_Ai will be
                                                          # replaced in the first iteration of pseudocode
                                                          # steps 1 to 4.
                                                          
    residue_matrix_R = PBN_matrix_P.copy()
    k = 1  # counts the number of iterations of pseudocode Steps 1 to 4

    chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
    chosen_entry_xi = choose_coefficient(residue_matrix_R, chosen_entry_pos_each_col, PBN_col_num)
    
    coefficient_list_xi.append(chosen_entry_xi)
    BN_matrices_list_Ai = chosen_entry_pos_each_col
    
    for col_counter in range(PBN_col_num):
        row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
        residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    zero_PBN_matrix = np.zeros((PBN_row_num, PBN_col_num), dtype=int)
    
    while (not np.array_equal(residue_matrix_R, zero_PBN_matrix)):
        k += 1
        chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
        chosen_entry_xi = choose_coefficient(residue_matrix_R, chosen_entry_pos_each_col, PBN_col_num)
    
        coefficient_list_xi.append(chosen_entry_xi)
        BN_matrices_list_Ai = np.append(BN_matrices_list_Ai, chosen_entry_pos_each_col, axis=0)
        
        for col_counter in range(PBN_col_num):
            row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
            residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    return k, coefficient_list_xi, BN_matrices_list_Ai

## The Greedy Entry Removal Algorithm (GER)

In [11]:
# The input first_B_vals_of_K of this function is a 1D numpy array of size
# (upper_bound_B + 1), and the entry at index zero of this array is to be ignored.
# The output of this function is a list.
def find_most_occurred_entries(first_B_vals_of_K, upper_bound_B):
    most_occurred_entries_v = []
    greatest_occurrence = -1
    
    for v in range(1, upper_bound_B + 1):
        if first_B_vals_of_K[v] > greatest_occurrence:
            greatest_occurrence = first_B_vals_of_K[v]
            most_occurred_entries_v = [v]
        elif first_B_vals_of_K[v] == greatest_occurrence:
            most_occurred_entries_v.append(v)
    
    return most_occurred_entries_v

In [12]:
def find_the_position_of_val_in_specific_col(residue_matrix_R, R_row_num, col, val):
    for row_counter in range(R_row_num):
        if val == residue_matrix_R[row_counter, col]:
            return row_counter

In [13]:
def all_entry_vals_greater_than_val(residue_matrix_R, R_row_num, col, val):
    output_list_of_entry_vals = []
    
    for row_counter in range(R_row_num):
        if residue_matrix_R[row_counter, col] > val:
            output_list_of_entry_vals.append(residue_matrix_R[row_counter, col])
    
    return output_list_of_entry_vals

In [14]:
# cumulative_occurrence_array is an array of size (1 + max_entry_M). The entry
# at index 0 is useless and should never be used.
def compute_selection_score(cumulative_occurrence_array, max_entry_M, score_parameter_z):
    selection_score = 0
    
    for v in range(1, max_entry_M + 1):
        if cumulative_occurrence_array[v] > 0:
            selection_score += score_parameter_z ** cumulative_occurrence_array[v]
    
    return selection_score

In [15]:
def choose_most_favourable_v_star(most_occurred_entries_v, selection_score_u_for_each_v):
    list_of_entries_with_max_score = []
    max_score_found_so_far = -1
    
    for v in most_occurred_entries_v:
        if max_score_found_so_far < selection_score_u_for_each_v[v]:
            max_score_found_so_far = selection_score_u_for_each_v[v]
            list_of_entries_with_max_score = [v]
        elif max_score_found_so_far == selection_score_u_for_each_v[v]:
            list_of_entries_with_max_score.append(v)
    
    return max(list_of_entries_with_max_score)

In [16]:
def GER_entry_choosing_procedure(residue_matrix_R, score_parameter_z):
    R_row_num = residue_matrix_R.shape[0]
    R_col_num = residue_matrix_R.shape[1]
    chosen_entry_pos_each_col = np.array([[-R_row_num - 1] * R_col_num])
    
    # Step 1
    max_entry_M = np.amax(residue_matrix_R)
    
    # Step 2
    # the col dimension should be max_entry_M + 1 because index starts at 0 but not 1.
    occurrence_count_Li = np.zeros((R_col_num, 1 + max_entry_M), dtype=int)
    occurrence_indication_Ki = np.zeros((R_col_num, 1 + max_entry_M), dtype=int)
    
    for col_counter in range(R_col_num):
        for row_counter in range(R_row_num):
            entry_val_a = residue_matrix_R[row_counter, col_counter]
            if entry_val_a > 0:
                occurrence_indication_Ki[col_counter, entry_val_a] = 1
                occurrence_count_Li[col_counter, entry_val_a] += 1
    
    # Step 3
    max_entry_each_col_Mi = np.amax(residue_matrix_R, axis=0)
    upper_bound_B = np.amin(max_entry_each_col_Mi)
    
    # Step 4
    occurrence_all_columns_K = np.sum(occurrence_indication_Ki, axis=0)
    
    # Step 5
    first_B_vals_of_K = occurrence_all_columns_K[0:upper_bound_B + 1] # the value at index 0 should be ignored.
    most_occurred_entries_v = find_most_occurred_entries(first_B_vals_of_K, upper_bound_B)
    # most_occurred_entries_v is a list.
    
    # Step 6
    chosen_entry_pos_each_col_for_each_v = {}
    indicator_array_for_selected_columns_for_each_v = {}
    occurrence_count_Li_for_each_v = {}
    occurrence_indication_Ki_for_each_v = {}
    cumulative_occurrence_indication_Ktoj_cumu_for_each_v = {}
    selection_score_u_for_each_v = {}
    
    for v in most_occurred_entries_v:
        chosen_entry_pos_each_col_for_each_v[v] = np.array([[-R_row_num - 1] * R_col_num])
        indicator_array_for_selected_columns_for_each_v[v] = np.zeros(R_col_num, dtype=int)
        occurrence_count_Li_for_each_v[v] = occurrence_count_Li.copy()
        occurrence_indication_Ki_for_each_v[v] = occurrence_indication_Ki.copy()
        cumulative_occurrence_indication_Ktoj_cumu_for_each_v[v] = np.zeros(1 + max_entry_M)
    
    for v in most_occurred_entries_v:
        for col_counter_i in range(R_col_num):
            if occurrence_indication_Ki_for_each_v[v][col_counter_i, v] == 1:
                q = find_the_position_of_val_in_specific_col(residue_matrix_R, R_row_num, col_counter_i, v)
                chosen_entry_pos_each_col_for_each_v[v][0, col_counter_i] = q
                indicator_array_for_selected_columns_for_each_v[v][col_counter_i] = 1
                
                occurrence_count_Li_for_each_v[v][col_counter_i, v] -= 1
                if occurrence_count_Li_for_each_v[v][col_counter_i, v] == 0:
                    occurrence_indication_Ki_for_each_v[v][col_counter_i, v] = 0
                cumulative_occurrence_indication_Ktoj_cumu_for_each_v[v] += \
                occurrence_indication_Ki_for_each_v[v][col_counter_i, :]
    
        for col_counter_i in range(R_col_num):
            if indicator_array_for_selected_columns_for_each_v[v][col_counter_i] == 0:
                list_of_candidate_entry_vals = \
                all_entry_vals_greater_than_val(residue_matrix_R, R_row_num, col_counter_i, v)
                
                occurrences_in_cumulative_Ktoj_cumu = {}
                for val in list_of_candidate_entry_vals:
                    occurrences_in_cumulative_Ktoj_cumu[val] = \
                    cumulative_occurrence_indication_Ktoj_cumu_for_each_v[v][val - v]
            
                greatest_occurrence = -1
                entry_with_greatest_occurrence = -1
            
                for val in list_of_candidate_entry_vals:
                    if greatest_occurrence < occurrences_in_cumulative_Ktoj_cumu[val]:
                        greatest_occurrence = occurrences_in_cumulative_Ktoj_cumu[val]
                        entry_with_greatest_occurrence = val
                
                chosen_entry_pos_each_col_for_each_v[v][0, col_counter_i] = \
                find_the_position_of_val_in_specific_col(residue_matrix_R, R_row_num, 
                                                         col_counter_i, entry_with_greatest_occurrence)
                
                indicator_array_for_selected_columns_for_each_v[v][col_counter_i] = 1
                
                occurrence_count_Li_for_each_v[v][col_counter_i, entry_with_greatest_occurrence] -= 1
                if occurrence_count_Li_for_each_v[v][col_counter_i, entry_with_greatest_occurrence] == 0:
                    occurrence_indication_Ki_for_each_v[v][col_counter_i, entry_with_greatest_occurrence] = 0
                
                occurrence_count_Li_for_each_v[v][col_counter_i, entry_with_greatest_occurrence - v] += 1
                if occurrence_count_Li_for_each_v[v][col_counter_i, entry_with_greatest_occurrence - v] == 1:
                    occurrence_indication_Ki_for_each_v[v][col_counter_i, 
                                                           entry_with_greatest_occurrence - v] = 1
                
                cumulative_occurrence_indication_Ktoj_cumu_for_each_v[v] += \
                occurrence_indication_Ki_for_each_v[v][col_counter_i, :]
        
        selection_score_u_for_each_v[v] = \
        compute_selection_score(cumulative_occurrence_indication_Ktoj_cumu_for_each_v[v], 
                                max_entry_M, score_parameter_z)
    
    # step 7
    most_favourable_v_star = \
    choose_most_favourable_v_star(most_occurred_entries_v, selection_score_u_for_each_v)
    
    chosen_entry_xi = most_favourable_v_star
    chosen_entry_pos_each_col = chosen_entry_pos_each_col_for_each_v[most_favourable_v_star]
                
    return chosen_entry_xi, chosen_entry_pos_each_col
    # chosen_entry_xi is of type int; chosen_entry_pos_each_col is a 1-by-PBN_col_num 2D numpy ndarray.

In [17]:
def GER(PBN_matrix_P, score_parameter_z):  # input: 2^n-by-2^n ndarray consisting of integers.
    PBN_row_num = PBN_matrix_P.shape[0]
    PBN_col_num = PBN_matrix_P.shape[1]
    coefficient_list_xi = []  # the coefficients of BN matrices to be output
    # the BN matrices to be output
    BN_matrices_list_Ai = np.array([[-1] * PBN_col_num])  # the first row of BN_matrices_list_Ai will be
                                                          # replaced in the first iteration of pseudocode
                                                          # steps 1 to 4.
                                                          
    residue_matrix_R = PBN_matrix_P.copy()
    k = 1  # counts the number of iterations of pseudocode Steps 1 to 4
    chosen_entry_xi, chosen_entry_pos_each_col = \
    GER_entry_choosing_procedure(residue_matrix_R, score_parameter_z)
    # chosen_entry_xi is of type int; chosen_entry_pos_each_col is a 1-by-PBN_col_num 2D numpy ndarray.
    
    coefficient_list_xi.append(chosen_entry_xi)
    BN_matrices_list_Ai = chosen_entry_pos_each_col
    
    for col_counter in range(PBN_col_num):
        row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
        residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    zero_PBN_matrix = np.zeros((PBN_row_num, PBN_col_num), dtype=int)
    
    while (not np.array_equal(residue_matrix_R, zero_PBN_matrix)):
        k += 1
        chosen_entry_xi, chosen_entry_pos_each_col = \
        GER_entry_choosing_procedure(residue_matrix_R,  score_parameter_z)
        # chosen_entry_xi is of type int; chosen_entry_pos_each_col is a 1-by-PBN_col_num 2D numpy ndarray.
    
        coefficient_list_xi.append(chosen_entry_xi)
        BN_matrices_list_Ai = np.append(BN_matrices_list_Ai, chosen_entry_pos_each_col, axis=0)
        
        for col_counter in range(PBN_col_num):
            row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
            residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    return k, coefficient_list_xi, BN_matrices_list_Ai

## The Single-DPre-SER2-GER algorithm (SDS2G)

In [18]:
def SDS2G(P, d_low, d_high, z):
    row_num, col_num = P.shape
    zero_matrix = np.zeros((row_num, col_num), dtype=int)
    K = math.inf
    x_list = []
    A_array = []
    optimal_d = 0
    Q_tilde_optimal_algo = ""
    R_tilde_optimal_algo = ""
    
    for d in range(d_low, d_high + 1):
        Q_tilde, R_tilde = division_preprocess(P, d)
        
        if ((not np.array_equal(Q_tilde, zero_matrix)) and (not np.array_equal(R_tilde, zero_matrix))):
            _, Q_tilde_x_list_SER2, Q_tilde_BNs_SER2 = SER_2(Q_tilde)
            _, Q_tilde_x_list_GER, Q_tilde_BNs_GER = GER(Q_tilde, z)
            _, R_tilde_x_list_SER2, R_tilde_BNs_SER2 = SER_2(R_tilde)
            _, R_tilde_x_list_GER, R_tilde_BNs_GER = GER(R_tilde, z)
            
            Q_tilde_x_list_SER2 = [i*d for i in Q_tilde_x_list_SER2]
            Q_tilde_x_list_GER = [i*d for i in Q_tilde_x_list_GER]
            
            merged_length, merged_x_list, merged_BNs =\
            merge_two_decompositions(Q_tilde_x_list_SER2, Q_tilde_BNs_SER2,
                                     R_tilde_x_list_SER2, R_tilde_BNs_SER2)
            if (merged_length < K):
                K = merged_length
                x_list = merged_x_list
                A_array = merged_BNs
                optimal_d = d
                Q_tilde_optimal_algo = "SER 2"
                R_tilde_optimal_algo = "SER 2"
                
            merged_length, merged_x_list, merged_BNs =\
            merge_two_decompositions(Q_tilde_x_list_SER2, Q_tilde_BNs_SER2,
                                     R_tilde_x_list_GER, R_tilde_BNs_GER)
            if (merged_length < K):
                K = merged_length
                x_list = merged_x_list
                A_array = merged_BNs
                optimal_d = d
                Q_tilde_optimal_algo = "SER 2"
                R_tilde_optimal_algo = "GER"
                
            merged_length, merged_x_list, merged_BNs =\
            merge_two_decompositions(Q_tilde_x_list_GER, Q_tilde_BNs_GER,
                                     R_tilde_x_list_SER2, R_tilde_BNs_SER2)
            if (merged_length < K):
                K = merged_length
                x_list = merged_x_list
                A_array = merged_BNs
                optimal_d = d
                Q_tilde_optimal_algo = "GER"
                R_tilde_optimal_algo = "SER 2"
            
            merged_length, merged_x_list, merged_BNs =\
            merge_two_decompositions(Q_tilde_x_list_GER, Q_tilde_BNs_GER,
                                     R_tilde_x_list_GER, R_tilde_BNs_GER)
            if (merged_length < K):
                K = merged_length
                x_list = merged_x_list
                A_array = merged_BNs
                optimal_d = d
                Q_tilde_optimal_algo = "GER"
                R_tilde_optimal_algo = "GER"
    
    return K, x_list, A_array, optimal_d, Q_tilde_optimal_algo, R_tilde_optimal_algo

## The input TPMs we are going to test

### $P_{1}$

In [19]:
P1 = np.array([[1599, 2501, 1271, 1964, 2501,    0,    0,  242],
               [ 184, 1618, 1906,    0,    0, 1906, 1308, 2501],
               [1308,    0,  224,  198,  173, 2494, 2469, 1895],
               [   0, 1308,    0, 1599, 1906, 1271, 2494, 2520],
               [   0,  198, 2494, 2480, 1290, 1567,    0, 1578],
               [2494, 2480,    0,    0, 1592, 2538, 1927, 1264],
               [1895,    0, 2538, 2469, 2538,  224, 1578,    0],
               [2520, 1895, 1567, 1290,    0,    0,  224,    0]])

### $P_{2}$

In [20]:
P2 = np.array([[2154, 2220,    0, 3741, 3214, 4677, 2208, 4640],
               [3208, 3183, 3209, 3440,  577, 8883, 8826,  351],
               [ 577, 9424, 8908, 2222, 8875, 3225, 3208, 8833],
               [  30, 3420,  400,  400,   19, 3402, 3420, 2192],
               [8859,    0,  584,    0, 2177,  369,  358, 3225],
               [4652,  389, 2177, 4684, 3397, 2154,  572,  609],
               [3451, 4684, 3397,    0, 4678,  610, 4678,   50],
               [ 389,    0, 4645, 8833,  383,    0,   50, 3420]])

### $P_{5}$ ($Q_{1}$)

In [21]:
Q1 = np.array([[2810,    0, 2080, 2676,  782,    0, 1840, 1832, 1981,  685, 1164,
                   0, 2806, 1824,    0,  331],
               [2660,    0, 1172, 2030,    0, 1175, 1219, 2886, 2061,    0, 2665,
                1607, 1255,  452,  999,    0],
               [   0, 2080, 1205,    0, 2825, 1172, 1007, 2649,  999, 2597, 1607,
                   0, 1180,    0, 1820, 2000],
               [   0, 2690,    0, 2597, 1156, 1112, 2495, 1205,    0,    0, 1236,
                 661, 2762, 1175, 2495, 1989],
               [1277,    0,  350, 1915,    0,  339, 1521,    0,    0, 1172, 1832,
                2080, 1989,    0, 2044,    0],
               [2511, 1879,    0,    0, 1236,    0,  669,    0, 1879, 2721, 2841,
                 355,    0, 1490, 1502, 1219],
               [1071, 2500, 2810,    0, 2030,  724,    0,  452,  685, 2810, 2080,
                2704, 1505,  710,    0, 1277],
               [2030, 1175, 2597, 1160, 1820, 1521, 1255, 1985, 1191,    0,  710,
                1832,    0,    0,  331,    0],
               [ 347, 1112,  696,  724, 2657, 1915, 2855, 1505, 2480, 1521, 1010,
                   0,    0, 2905, 2676, 1071],
               [1981,    0,  991,  331,    0, 2556, 2645, 2016, 1505, 1989,    0,
                1175, 2030, 1981, 1180, 2676],
               [1494, 1236, 2044, 2830,  339, 2830, 1981,  760, 2657, 1040, 2480,
                2061, 2539, 2657,  741, 1915],
               [ 724, 2822, 2649,  999, 1521,    0,  411,    0,    0, 1835,    0,
                1040,    0, 2500,    0,    0],
               [1255,  661,    0,    0,    0, 2641,    0, 1156,  452, 1219,  331,
                1160, 1026, 1191, 1255, 1502],
               [   0, 1521, 1566, 1997, 1015, 1985,    0, 2500, 2905, 1981, 2044,
                2511,  335, 2061, 2927,  685],
               [   0,  339, 1840, 1175, 2539, 2030, 2102,    0,    0,    0,    0,
                2814, 1832,    0, 2030, 2810],
               [1840, 1985,    0, 1566, 2080,    0,    0, 1054, 1205,  430,    0,
                   0,  741, 1054,    0, 2525]])

### $P_{6}$ ($Q_{2}$)

In [22]:
Q2 = np.array([[    0,  5750, 13587,  3407, 13616, 18379,     0, 12038, 13574,
                13566,  3217,  6668,     0, 18387, 13577,  4819],
               [ 4813,  9470,  2153,  3260,  1660, 13992,  3352, 18385,  2137,
                  829,  3751,  1617,  7117,   806,  3325,   143],
               [  143,  4590,  3334,  3769,  9511, 15837, 18533,  2269,  2292,
                11971, 13597,  2003,     0,  3733,  6668,     0],
               [15173,  4851,   825,  9453,  4833,  2311,  4838,  6683,  2279,
                 3734,  3343,  2271,  9601,  4789,  2262,  6658],
               [  844,     0,  2279, 14398,  2272,  4797,  4555,  2271, 15305,
                 2059,  4788,  2128, 13593,  2272,  2004,  2178],
               [ 3792, 11989, 11978,   154,  2270,  9461,  8877,   158,  4799,
                 6658,  6695,  2325,  2055,  1646, 12038,  3788],
               [ 2282, 13567,  6695,     0,   816,     0,  3193,  1607, 18422,
                 4856, 18362,  3755,  4852,  2149,  3755,  2306],
               [ 3334,   197,  1996,  2012, 12003,   134,     0,  3213,  3213,
                 2288,  2045, 18387,  3263, 11995,  9453,  2028],
               [13574,  3202,  1664, 14256,  1995,  3370,     0,   843,  1673,
                  158,  1605,  9468, 13576,  2288,  4821, 11995],
               [ 1627, 20507, 18404, 18381,  3326,  1664, 17301,  3725,  9476,
                 1646,  9453,  3201,  2139,  3237,  3237,  3352],
               [ 6683,     0,  3256,  3907,  3213,  3732,  9488, 13630,   178,
                 3203, 11981, 11971,     0,   201,  2282,     0],
               [11731,  3396,  2294,     0, 18381,   862,   816,  2045,  7512,
                 2279,  2294,  3336,  6651, 13630,   862,  4017],
               [ 2146,  1629,  9474,     0,  6681,  3213, 12043,  4815,  2020,
                18399,   799,   160,  3067,  2005,  1646, 11040],
               [ 2045,  7457,  4799,  6661,   143,     0,  2005,  9461,     0,
                 9476,  2325,  4856,     0,  3356, 18381,  2272],
               [18418,     0,   143,  6947,  2153,  2192,     0,  2128,     0,
                 2149,  2196,   848,  2306,  9443,   141, 13587],
               [    0,     0,  3724,     0,  3732,  6661,  1604,  3334,  3725,
                 3334,   154, 13611, 18385,  6668,  2153, 18422]])

# Numerical Experiments (SDS2G)

### $P_{1}$

In [44]:
P = P1
d_low = 30
d_high = 200
z = 10

time_before_execution = datetime.datetime.now()
K, x_list, A_array, optimal_d, Q_tilde_optimal_algo, R_tilde_optimal_algo = SDS2G(P, d_low, d_high, z)
time_after_execution = datetime.datetime.now()

execution_duration = time_after_execution - time_before_execution

print(K)
print(x_list)
print(A_array)
print(optimal_d)
print(Q_tilde_optimal_algo)
print(R_tilde_optimal_algo)
print(execution_duration)

12
[2460, 2460, 1886, 1558, 1230, 164, 78, 60, 41, 34, 20, 9]
[[5 0 4 4 0 2 2 1]
 [7 5 6 6 6 5 3 3]
 [6 7 1 0 3 1 5 2]
 [0 1 7 3 5 4 6 4]
 [2 3 0 7 4 3 1 5]
 [1 4 2 2 2 6 7 0]
 [2 3 6 0 6 5 1 0]
 [7 1 2 7 4 6 7 3]
 [0 0 0 3 0 3 5 1]
 [5 4 4 2 5 2 3 5]
 [1 5 1 4 3 1 6 4]
 [6 7 7 6 2 4 2 2]]
82
SER 2
SER 2
0:00:02.977563


In [35]:
row_dim = 8
col_dim = 8

P1 == add_up_a_decomposition(x_list, A_array, row_dim, col_dim)

array([[ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True]])

### $P_{2}$

In [46]:
P = P2
d_low = 30
d_high = 200
z = 10

time_before_execution = datetime.datetime.now()
K, x_list, A_array, optimal_d, Q_tilde_optimal_algo, R_tilde_optimal_algo = SDS2G(P, d_low, d_high, z)
time_after_execution = datetime.datetime.now()

execution_duration = time_after_execution - time_before_execution

print(K)
print(x_list)
print(A_array)
print(optimal_d)
print(Q_tilde_optimal_algo)
print(R_tilde_optimal_algo)
print(execution_duration)

15
[4633, 3390, 2147, 339, 8814, 3164, 565, 61, 12, 45, 30, 19, 50, 44, 7]
[[5 6 7 5 6 0 6 0]
 [6 3 6 1 5 3 3 7]
 [0 0 5 2 4 5 0 3]
 [7 5 3 3 7 4 4 1]
 [4 2 2 7 2 1 1 2]
 [1 1 1 0 0 2 2 4]
 [2 2 4 0 1 6 5 5]
 [6 0 3 3 2 2 0 4]
 [2 0 7 0 1 3 1 1]
 [4 2 1 2 6 6 6 3]
 [3 3 5 2 4 4 3 7]
 [5 1 4 7 3 1 4 2]
 [7 5 2 1 0 1 7 6]
 [1 6 2 5 7 0 2 5]
 [0 6 6 5 5 5 5 0]]
113
GER
GER
0:00:04.361493


In [24]:
row_dim = 8
col_dim = 8

P2 == add_up_a_decomposition(x_list, A_array, row_dim, col_dim)

array([[ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True]])

### $Q_{1}$

In [47]:
Q = Q1
d_low = 30
d_high = 200
z = 10

time_before_execution = datetime.datetime.now()
K, x_list, A_array, optimal_d, Q_tilde_optimal_algo, R_tilde_optimal_algo = SDS2G(Q, d_low, d_high, z)
time_after_execution = datetime.datetime.now()

execution_duration = time_after_execution - time_before_execution

print(K)
print(x_list)
print(A_array)
print(optimal_d)
print(Q_tilde_optimal_algo)
print(R_tilde_optimal_algo)
print(execution_duration)

24
[2805, 2640, 2475, 1980, 1980, 1815, 1485, 1155, 1155, 990, 660, 330, 122, 100, 81, 64, 50, 36, 25, 20, 17, 9, 5, 1]
[[ 0 11  6 10  2 10  8  1 13  6  5 14  0  8 13 14]
 [ 1  3 11  0  8 12  9  2 10  5  1  6  3 10  8  9]
 [ 5  6  7  3 14  9  3 13  8  2 10 13 10 11  3 15]
 [ 7  2  0  1  6 13 10  7  0  9  6  4  4  9  4  2]
 [ 9 15 10 13 15 14 14  9  1 13 13 10  9 13 14  3]
 [15  5 14  4  7  8  0  0  5 11  4  7 14  0  2 10]
 [10 13 13 15 11  7  4  8  9  8  2  1  6  5  5 12]
 [ 4  7  1  7  3  1  1  3  7  4  0  9  1  3  9  5]
 [12 10  2 14  5  2  7 12 15 12  3 12  2 12 12  6]
 [ 6  8  9 11 13  3  2 15  2 10  8 11 12 15  1  8]
 [11 12  8  8  0  6  5 10  6  0  7  3 15  6 10 13]
 [ 8 14  4  9 10  4 11  6 12 15 12  5 13  1  7  0]
 [ 4  8  7  3  0  3 14  6 12  2  2  1  3  1 13  6]
 [12  2  0  4 15  8  7 10 13 15  6  4  1  8 12 10]
 [ 6 10 13 15  5  9 11  1  1  5  3 10 15 13 10  8]
 [11  5 10  8 14  6  1 15  5 12 13  6 10 15  4  5]
 [ 7  3  2  1  6 14  8  3 15 10  7 11  9  6 14 15]
 [ 5 13  8  0

In [26]:
row_dim = 16
col_dim = 16

Q1 == add_up_a_decomposition(x_list, A_array, row_dim, col_dim)

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  Tru

### $Q_{2}$

In [48]:
Q = Q2
d_low = 30
d_high = 200
z = 10

time_before_execution = datetime.datetime.now()
K, x_list, A_array, optimal_d, Q_tilde_optimal_algo, R_tilde_optimal_algo = SDS2G(Q, d_low, d_high, z)
time_after_execution = datetime.datetime.now()

execution_duration = time_after_execution - time_before_execution

print(K)
print(x_list)
print(A_array)
print(optimal_d)
print(Q_tilde_optimal_algo)
print(R_tilde_optimal_algo)
print(execution_duration)

34
[4788, 3325, 3724, 13566, 1995, 133, 9443, 1596, 2261, 3192, 2261, 11970, 6650, 798, 2128, 11438, 4256, 2527, 133, 10, 68, 11, 31, 27, 8, 25, 50, 21, 1, 45, 33, 64, 18, 9]
[[ 1  3 13  9  3  4  3 12  5  6  4 13  6  3  8  0]
 [ 7 11  2  0  9  8  1 15  4 15  3 11  1 13  1  9]
 [ 5  0 15  2 15 10  9  9 15  3  1  6  1  2  6  5]
 [ 8  6  0  9  0  2  9 10  0  0  2 15  4 11  0 14]
 [13  0  7  7  8 14 13 11 12  4  7  2  5 12  4  7]
 [ 2  7 14  5 13  7  2  5 10  8 15 12  3 10 14  1]
 [11  1 12  3  2  5 10 13  9 13  9  8  3 14  7 12]
 [ 9 12  8 10  1  9 15  6  8  9  8  1  8  5 12 12]
 [ 6  2  4 10  4  2  4  2  2  7 11  3 14  4  3  6]
 [ 3  8 10  1 10 12  6  7  7 10  0  9  7  9  9 11]
 [11  2 11  8  5  3  4  4  3 11 13  5 12  8 10 13]
 [ 3  5  5  8  7  0 12  0  4  2 10 10  8  7  5  8]
 [10 13  6 13 12 15  5  3 11  5  5  0 11 15  2  3]
 [ 4 13  3  4  6 11 11  8 11  1 12 14 12  1 11 11]
 [12  9  1  4 14  0  5 14  1 14 14  4  9  6 15  4]
 [14  9  9  4 11  1  2  1  6 12  6  7 15  0 13 15]
 [14  9  

In [28]:
row_dim = 16
col_dim = 16

Q2 == add_up_a_decomposition(x_list, A_array, row_dim, col_dim)

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  Tru

# Numerical Experiments (GER)

### $P_{1}$

In [66]:
lengths_of_decompositions = []
execution_times = []

for z_parameter in range(2, 21):
    time_before_execution = datetime.datetime.now()
    k, coefficient_list_xi, BN_matrices_list_Ai = GER(P1, z_parameter)
    time_after_execution = datetime.datetime.now()
    
    execution_duration = time_after_execution - time_before_execution
    lengths_of_decompositions.append(k)
    execution_times.append(execution_duration)

print(lengths_of_decompositions)
print(max(execution_times))
print(sum(execution_times, datetime.timedelta()) / len(execution_times))
print(min(execution_times))

[22, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]
0:00:00.051339
0:00:00.031439
0:00:00.024683


### $P_{2}$

In [70]:
lengths_of_decompositions = []
execution_times = []

for z_parameter in range(2, 21):
    time_before_execution = datetime.datetime.now()
    k, coefficient_list_xi, BN_matrices_list_Ai = GER(P2, z_parameter)
    time_after_execution = datetime.datetime.now()
    
    execution_duration = time_after_execution - time_before_execution
    lengths_of_decompositions.append(k)
    execution_times.append(execution_duration)

print(lengths_of_decompositions)
print(max(execution_times))
print(sum(execution_times, datetime.timedelta()) / len(execution_times))
print(min(execution_times))

[30, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33]
0:00:00.270271
0:00:00.183380
0:00:00.151033


### $Q_{1}$

In [71]:
lengths_of_decompositions = []
execution_times = []

for z_parameter in range(2, 21):
    time_before_execution = datetime.datetime.now()
    k, coefficient_list_xi, BN_matrices_list_Ai = GER(Q1, z_parameter)
    time_after_execution = datetime.datetime.now()
    
    execution_duration = time_after_execution - time_before_execution
    lengths_of_decompositions.append(k)
    execution_times.append(execution_duration)

print(lengths_of_decompositions)
print(max(execution_times))
print(sum(execution_times, datetime.timedelta()) / len(execution_times))
print(min(execution_times))

[60, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66]
0:00:00.356680
0:00:00.242593
0:00:00.199324


### $Q_{2}$

In [72]:
lengths_of_decompositions = []
execution_times = []

for z_parameter in range(2, 21):
    time_before_execution = datetime.datetime.now()
    k, coefficient_list_xi, BN_matrices_list_Ai = GER(Q2, z_parameter)
    time_after_execution = datetime.datetime.now()
    
    execution_duration = time_after_execution - time_before_execution
    lengths_of_decompositions.append(k)
    execution_times.append(execution_duration)

print(lengths_of_decompositions)
print(max(execution_times))
print(sum(execution_times, datetime.timedelta()) / len(execution_times))
print(min(execution_times))

[83, 90, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92, 92]
0:00:01.838730
0:00:01.407377
0:00:01.178561


# Numerical Experiments (SER 1)

### $P_{1}$

In [57]:
time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(P1)
time_after_execution = datetime.datetime.now()

execution_duration = time_after_execution - time_before_execution

print(execution_duration)
print(k)
print(coefficient_list_xi)
print(BN_matrices_list_Ai)

0:00:00.002299
41
[173, 184, 198, 198, 224, 224, 224, 242, 1264, 621, 631, 631, 635, 636, 598, 472, 561, 375, 267, 279, 280, 102, 106, 123, 180, 93, 111, 74, 77, 40, 43, 36, 41, 9, 9, 10, 10, 5, 6, 2, 6]
[[7 0 6 4 2 5 3 3]
 [1 5 4 6 6 2 2 1]
 [5 4 6 4 0 5 3 3]
 [7 0 4 2 6 2 2 1]
 [5 5 2 6 0 5 3 3]
 [7 0 6 4 6 6 2 1]
 [5 5 4 6 0 2 7 3]
 [7 0 6 0 6 5 5 0]
 [6 7 1 4 3 1 3 5]
 [5 5 4 4 0 2 2 1]
 [6 0 6 6 6 5 5 2]
 [7 7 7 0 5 4 6 3]
 [0 1 0 3 4 3 3 4]
 [2 3 0 7 0 2 1 1]
 [5 5 4 6 0 5 2 2]
 [7 0 6 0 6 5 5 3]
 [0 0 7 3 5 4 6 4]
 [2 1 7 7 4 1 1 2]
 [5 3 4 0 3 1 2 1]
 [7 5 1 7 6 3 5 3]
 [0 1 6 6 4 2 6 4]
 [5 3 4 3 5 4 2 4]
 [7 5 1 0 3 3 6 1]
 [0 1 6 6 6 2 5 3]
 [2 3 4 3 5 4 5 2]
 [5 5 1 0 3 4 1 1]
 [7 1 6 6 6 3 2 2]
 [5 5 1 0 6 2 1 3]
 [2 5 4 3 3 2 2 1]
 [2 3 1 6 5 3 1 3]
 [5 1 4 0 3 3 1 1]
 [7 3 6 0 5 2 2 3]
 [5 1 1 6 3 2 2 1]
 [5 3 6 3 5 3 1 3]
 [7 3 1 3 5 3 1 3]
 [7 1 6 3 5 3 1 3]
 [7 3 6 3 3 3 2 3]
 [7 3 6 6 3 3 1 1]
 [7 3 6 3 5 3 1 3]
 [7 3 6 6 5 3 1 3]
 [7 3 6 6 5 3 1 1]]


### $P_{2}$

In [58]:
time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(P2)
time_after_execution = datetime.datetime.now()

execution_duration = time_after_execution - time_before_execution

print(execution_duration)
print(k)
print(coefficient_list_xi)
print(BN_matrices_list_Ai)

0:00:00.003136
51
[19, 30, 50, 50, 351, 358, 369, 383, 389, 389, 400, 400, 572, 577, 577, 584, 609, 610, 2154, 1587, 1882, 1515, 1520, 672, 688, 700, 1165, 503, 524, 528, 529, 450, 313, 361, 292, 253, 187, 145, 155, 110, 58, 70, 86, 42, 45, 13, 14, 15, 9, 8, 10]
[[4 2 2 7 3 1 1 2]
 [3 2 2 7 2 1 1 2]
 [4 2 2 7 2 1 7 2]
 [4 2 2 7 2 1 1 6]
 [4 2 2 7 2 1 1 1]
 [4 2 2 7 2 1 4 2]
 [4 2 2 7 2 4 1 2]
 [4 2 2 7 7 1 1 2]
 [7 2 2 7 2 1 1 2]
 [4 5 2 7 2 1 1 2]
 [4 2 3 7 2 1 1 2]
 [4 2 2 3 2 1 1 2]
 [4 2 2 7 2 1 5 2]
 [2 2 2 7 2 1 1 2]
 [4 2 2 7 1 1 1 2]
 [4 2 4 5 2 0 1 0]
 [5 6 7 7 6 1 6 5]
 [4 2 2 5 2 6 1 2]
 [0 6 7 0 6 0 6 0]
 [5 2 2 0 2 1 1 2]
 [4 3 7 7 5 3 3 7]
 [6 1 6 5 5 2 2 4]
 [1 0 1 1 0 3 0 3]
 [5 2 2 2 4 5 1 3]
 [6 6 5 5 2 1 0 2]
 [4 0 6 1 6 0 6 0]
 [4 1 1 7 0 2 2 4]
 [5 1 2 2 4 5 3 7]
 [1 3 1 5 2 1 1 2]
 [5 2 5 1 6 0 2 0]
 [6 6 6 2 0 5 6 7]
 [1 3 2 5 4 5 3 2]
 [5 2 5 5 2 1 1 0]
 [6 6 6 1 6 0 6 0]
 [1 2 6 7 2 1 1 4]
 [5 3 5 2 4 2 3 4]
 [5 6 2 7 2 0 3 7]
 [1 2 5 1 6 1 3 2]
 [6 3 2 2 2 2 6

### $Q_{1}$

In [59]:
time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(Q1)
time_after_execution = datetime.datetime.now()

execution_duration = time_after_execution - time_before_execution

print(execution_duration)
print(k)
print(coefficient_list_xi)
print(BN_matrices_list_Ai)

0:00:00.018212
119
[331, 331, 331, 331, 335, 339, 339, 339, 347, 350, 355, 411, 430, 452, 452, 452, 661, 661, 669, 685, 685, 668, 557, 563, 563, 587, 404, 423, 410, 379, 355, 330, 361, 322, 313, 299, 304, 265, 271, 163, 171, 183, 179, 155, 165, 158, 155, 150, 134, 137, 87, 94, 97, 80, 86, 80, 77, 74, 72, 65, 57, 50, 47, 43, 40, 40, 40, 30, 31, 27, 28, 22, 26, 20, 21, 22, 13, 19, 13, 15, 13, 14, 13, 12, 11, 10, 10, 6, 7, 7, 6, 6, 5, 6, 3, 4, 3, 4, 3, 3, 3, 2, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[[ 0 11  6 ...  8 13 14]
 [ 1  3 11 ... 10  8  9]
 [ 5  6  7 ...  8  7 15]
 ...
 [12 15  6 ...  9  8  6]
 [15 11 11 ... 13 10  8]
 [15 15 14 ... 15 13 10]]


### $Q_{2}$

In [60]:
time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_1(Q2)
time_after_execution = datetime.datetime.now()

execution_duration = time_after_execution - time_before_execution

print(execution_duration)
print(k)
print(coefficient_list_xi)
print(BN_matrices_list_Ai)

0:00:00.016908
164
[134, 141, 143, 143, 143, 143, 154, 154, 158, 158, 160, 178, 197, 201, 799, 806, 816, 816, 825, 829, 843, 844, 848, 862, 862, 1604, 1605, 1607, 1617, 1627, 1629, 1646, 1646, 1646, 1660, 1664, 1664, 1673, 1995, 1996, 2003, 2004, 2005, 2005, 2012, 2020, 2028, 2045, 2045, 1689, 1654, 1671, 1530, 1450, 1327, 979, 1144, 1125, 1082, 1057, 1035, 1058, 985, 774, 790, 752, 698, 639, 508, 515, 541, 540, 526, 490, 507, 399, 357, 395, 330, 316, 272, 312, 258, 260, 264, 247, 252, 173, 179, 178, 144, 155, 153, 143, 144, 131, 133, 71, 122, 71, 75, 79, 79, 79, 68, 73, 66, 62, 49, 50, 38, 41, 42, 37, 38, 30, 33, 19, 24, 22, 24, 21, 21, 17, 19, 11, 15, 11, 11, 11, 11, 9, 9, 7, 6, 7, 6, 6, 5, 5, 4, 4, 3, 3, 3, 2, 3, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[[14  9  9 ...  0 13 15]
 [14  9  9 ...  0 14 15]
 [ 2  9  9 ...  0 13 15]
 ...
 [11 13  4 ...  9 10 11]
 [12 11  6 ... 13 13 14]
 [12 13 15 ... 14 15 15]]


# Numerical Experiments (SER 2)

### $P_{1}$

In [61]:
time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_2(P1)
time_after_execution = datetime.datetime.now()

execution_duration = time_after_execution - time_before_execution

print(execution_duration)
print(k)
print(coefficient_list_xi)
print(BN_matrices_list_Ai)

0:00:00.001001
16
[2480, 2469, 1895, 1567, 1264, 173, 44, 32, 25, 14, 11, 11, 7, 4, 3, 1]
[[7 0 6 4 6 5 3 3]
 [5 5 4 6 0 2 2 1]
 [6 7 1 0 3 1 5 2]
 [0 1 7 3 5 4 6 4]
 [2 3 0 7 4 3 1 5]
 [1 4 2 2 2 6 7 0]
 [2 1 6 0 6 5 7 0]
 [7 3 2 3 0 6 1 3]
 [0 4 4 7 4 2 5 1]
 [5 0 2 0 5 6 3 0]
 [1 3 6 2 6 5 1 0]
 [5 5 1 2 3 1 6 4]
 [7 0 0 0 5 3 5 3]
 [0 1 2 0 5 6 7 1]
 [0 1 6 2 6 5 7 1]
 [7 3 2 7 4 6 1 3]]


### $P_{2}$

In [62]:
time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_2(P2)
time_after_execution = datetime.datetime.now()

execution_duration = time_after_execution - time_before_execution

print(execution_duration)
print(k)
print(coefficient_list_xi)
print(BN_matrices_list_Ai)

0:00:00.002047
22
[8826, 4640, 3397, 3183, 2154, 400, 344, 172, 49, 39, 33, 25, 23, 11, 6, 5, 5, 3, 2, 1, 1, 1]
[[4 2 2 7 2 1 1 2]
 [5 6 7 5 6 0 6 0]
 [6 3 6 0 5 3 3 7]
 [1 1 1 1 0 2 2 4]
 [0 0 5 2 4 5 0 3]
 [2 2 4 3 1 6 5 5]
 [7 5 3 0 7 4 4 1]
 [2 2 4 1 1 6 5 5]
 [6 0 2 1 2 1 0 6]
 [7 5 3 2 7 2 7 4]
 [4 6 2 5 6 6 6 3]
 [3 2 1 1 0 0 2 5]
 [1 3 5 2 4 4 3 7]
 [5 0 3 1 3 0 4 5]
 [7 6 4 5 3 1 7 1]
 [2 0 3 7 0 3 0 2]
 [3 5 4 2 1 6 6 3]
 [6 6 7 5 6 2 7 4]
 [1 6 7 5 3 1 4 2]
 [6 0 1 7 6 4 7 1]
 [5 2 3 2 0 0 4 5]
 [6 5 4 7 6 4 7 6]]


### $Q_{1}$

In [63]:
time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_2(Q1)
time_after_execution = datetime.datetime.now()

execution_duration = time_after_execution - time_before_execution

print(execution_duration)
print(k)
print(coefficient_list_xi)
print(BN_matrices_list_Ai)

0:00:00.002020
38
[2806, 2641, 2480, 1989, 1981, 1820, 1490, 1175, 1156, 991, 661, 331, 102, 80, 61, 49, 30, 20, 19, 16, 15, 15, 14, 11, 10, 8, 4, 4, 4, 4, 3, 3, 2, 1, 1, 1, 1, 1]
[[ 0 11  6 10  2 10  8  1 13  6  5 14  0  8 13 14]
 [ 1  3 11  0  8 12  9  2 10  5  1  6  3 10  8  9]
 [ 5  6  7  3 14  9  3 13  8  2 10 13 10 11  3 15]
 [ 7  2  0  1 15 14 14  9  1  9  6  4  9 13  4  2]
 [ 9 15 10 13  6 13 10  7  0 13 13 10  4  9 14  3]
 [15  5 14  4  7  8  0  0  5 11  4  7 14  0  2 10]
 [10 13 13 15 11  7  4  8  9  8  2  1  6  5  5 12]
 [ 4 10  2 14  5  1  7  3 15 12  3  9  1 12 12  6]
 [12  7  1  7  3  2  1 12  7  4  0 12  2  3  9  5]
 [ 6  8  9 11 13  3  2 15  2 10  8 11 12 15  1  8]
 [11 12  8  8  0  6  5 10  6  0  7  3 15  6 10 13]
 [ 8 14  4  9 10  4 11  6 12 15 12  5 13  1  7  0]
 [ 4  8  7  3  0  3 14  6 12  2  2  1  3  1 13  6]
 [12  2  0  4 15  8  7 10 13 15  6  4  1  8 10 10]
 [ 6 10 13 15  5  9 11  1  1  5 13 10 15 13 12  8]
 [11  5 10  8 14  6  1 15  5 10  3  6 10 15  4  5]
 [ 7

### $Q_{2}$

In [64]:
time_before_execution = datetime.datetime.now()
k, coefficient_list_xi, BN_matrices_list_Ai = SER_2(Q2)
time_after_execution = datetime.datetime.now()

execution_duration = time_after_execution - time_before_execution

print(execution_duration)
print(k)
print(coefficient_list_xi)
print(BN_matrices_list_Ai)

0:00:00.002406
59
[18362, 13566, 11971, 9443, 6658, 4788, 3724, 3213, 2288, 2271, 2137, 1629, 1377, 1064, 913, 651, 541, 486, 376, 165, 148, 132, 123, 110, 82, 63, 46, 36, 35, 25, 25, 23, 19, 15, 13, 11, 10, 10, 9, 7, 6, 4, 4, 3, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[[14  9  9  9 11  0  2  1  6 12  6  7 15  0 13 15]
 [ 3  6  0  4  0  2  9 10  4  0  2 15  4 11  0 14]
 [ 8  5  5  8  7  1 12  0  0  2 10 10  8  7  5  8]
 [11  1 12  3  2  5 10 13  9 13  9  8  3 14  7 12]
 [10 13  6 14 12 15  5  3 11  5  5  0  1 15  2  3]
 [ 1  0 13 13  3  4  3 12  5  6  4 13 11  3  8  0]
 [ 5  3 15 10 15 10  4  9 15  3  1  6  6  2  6 11]
 [ 7  2  2  2  9  8  9 15  7 15  3 11  7 13  1  5]
 [11 11 10  0 10 12  1  7  2 10  0  9 12  9  9  9]
 [ 6  8 11  1  4  3  6  4  3  7 13  5 14  8 10  6]
 [12  9  4  8  5  2  5  2  1 11 11  3  9  4  3 13]
 [13 12  1  7 14 14 13 14 12 14 14  4  5  6 15  4]
 [ 9  2  7 13  8  1 15 11  4  4  7  2 11 12  4  7]
 [ 3  3  8  0  1  9  1  6  8  9  8  1  8  5 12 12]
 [ 8 11 10 